# KITTI Dataset Visualizing

https://navoshta.com/kitti-lidar/

tracklets parsing

## 1. KITTI Data 다운로드 

![](https://i.imgur.com/SSP9eG4.png)

[Raw Data](http://www.cvlibs.net/datasets/kitti/raw_data.php)
- Length: 114 frames (00:11 minutes)
- Image resolution: 1392 x 512 pixels
- Labels: 12 Cars, 0 Vans, 0 Trucks, 0 Pedestrians, 0 Sitters, 2 Cyclists, 1 Trams, 0 Misc

[주의]
- This package assumes that you have also downloaded the calibration data associated with the sequences you want to work on (these are separate files from the sequences themselves), and that the directory structure is unchanged from the original structure laid out in the KITTI zip files.

In [18]:
!wget http://kitti.is.tue.mpg.de/kitti/raw_data/2011_09_26_drive_0001/2011_09_26_drive_0001_sync.zip
!wget http://kitti.is.tue.mpg.de/kitti/raw_data/2011_09_26_calib.zip
!unzip 2011_09_26_drive_0001_sync.zip
!unzip 2011_09_26_calib.zip

# 추가적으로 다운로드 
!wget http://kitti.is.tue.mpg.de/kitti/raw_data/2011_09_26_drive_0001/2011_09_26_drive_0001_tracklets.zip
!unzip 2011_09_26_drive_0001_tracklets.zip

--2017-09-26 11:23:33--  http://kitti.is.tue.mpg.de/kitti/raw_data/2011_09_26_drive_0001/2011_09_26_drive_0001_sync.zip
Resolving kitti.is.tue.mpg.de (kitti.is.tue.mpg.de)... 192.124.27.130
Connecting to kitti.is.tue.mpg.de (kitti.is.tue.mpg.de)|192.124.27.130|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 458643963 (437M) [application/zip]
Saving to: ‘2011_09_26_drive_0001_sync.zip.1’

2011_09_26_drive_00 100%[===================>] 437.40M  8.71MB/s    in 52s     

2017-09-26 11:24:26 (8.48 MB/s) - ‘2011_09_26_drive_0001_sync.zip.1’ saved [458643963/458643963]

--2017-09-26 11:24:26--  http://kitti.is.tue.mpg.de/kitti/raw_data/2011_09_26_calib.zip
Resolving kitti.is.tue.mpg.de (kitti.is.tue.mpg.de)... 192.124.27.130
Connecting to kitti.is.tue.mpg.de (kitti.is.tue.mpg.de)|192.124.27.130|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4068 (4.0K) [application/zip]
Saving to: ‘2011_09_26_calib.zip.1’

2011_09_26_calib.zi 100%[===========

## 2. pykitti 설치 및 Import  

In [2]:
# https://github.com/utiasSTARS/pykitti
!pip install pykitti
import pykitti

## From Source 
# git clone https://github.com/utiasSTARS/pykitti.git
# cd pykitti
# python setup.py install

ImportError: No module named pykitti

## 3. `parseTrackletXML.py` 다운로드 및 Import
- Methods for parsing tracklets (e.g. dataset labels), originally created by Christian Herdtweck.
- https://github.com/navoshta/KITTI-Dataset/tree/master/source

In [13]:
! git clone https://github.com/navoshta/KITTI-Dataset.git
! cp KITTI-Dataset/source/parseTrackletXML.py ./
import parseTrackletXML as xmlParser

### 3.1 load_tracklets_for_frames() 정의


In [6]:
def load_tracklets_for_frames(n_frames, xml_path):
    """
    Loads dataset labels also referred to as tracklets, saving them individually for each frame.

    Parameters
    ----------
    n_frames    : Number of frames in the dataset.
    xml_path    : Path to the tracklets XML.

    Returns
    -------
    Tuple of dictionaries with integer keys corresponding to absolute frame numbers and arrays as values. First array
    contains coordinates of bounding box vertices for each object in the frame, and the second array contains objects
    types as strings.
    """
    tracklets = xmlParser.parseXML(xml_path)

    frame_tracklets = {}
    frame_tracklets_types = {}
    for i in range(n_frames):
        frame_tracklets[i] = []
        frame_tracklets_types[i] = []

    # loop over tracklets
    for i, tracklet in enumerate(tracklets):
        # this part is inspired by kitti object development kit matlab code: computeBox3D
        h, w, l = tracklet.size
        # in velodyne coordinates around zero point and without orientation yet
        trackletBox = np.array([
            [-l / 2, -l / 2, l / 2, l / 2, -l / 2, -l / 2, l / 2, l / 2],
            [w / 2, -w / 2, -w / 2, w / 2, w / 2, -w / 2, -w / 2, w / 2],
            [0.0, 0.0, 0.0, 0.0, h, h, h, h]
        ])
        # loop over all data in tracklet
        for translation, rotation, state, occlusion, truncation, amtOcclusion, amtBorders, absoluteFrameNumber in tracklet:
            # determine if object is in the image; otherwise continue
            if truncation not in (xmlParser.TRUNC_IN_IMAGE, xmlParser.TRUNC_TRUNCATED):
                continue
            # re-create 3D bounding box in velodyne coordinate system
            yaw = rotation[2]  # other rotations are supposedly 0
            assert np.abs(rotation[:2]).sum() == 0, 'object rotations other than yaw given!'
            rotMat = np.array([
                [np.cos(yaw), -np.sin(yaw), 0.0],
                [np.sin(yaw), np.cos(yaw), 0.0],
                [0.0, 0.0, 1.0]
            ])
            cornerPosInVelo = np.dot(rotMat, trackletBox) + np.tile(translation, (8, 1)).T
            frame_tracklets[absoluteFrameNumber] = frame_tracklets[absoluteFrameNumber] + [cornerPosInVelo]
            frame_tracklets_types[absoluteFrameNumber] = frame_tracklets_types[absoluteFrameNumber] + [tracklet.objectType]

    return (frame_tracklets, frame_tracklets_types)

## 3. KITTI data Load
- The 'frames' argument is optional - default: None, which loads the whole dataset.
- Calibration and timestamp data are read automatically. 
- Other sensor data (cameras, IMU, Velodyne) are available via properties 
- that create generators when accessed.

In [30]:
n_frames = len(list(dataset.velo))

frame_tracklets = {}
frame_tracklets_types = {}
for i in range(n_frames):
    frame_tracklets[i] = []
    frame_tracklets_types[i] = []



In [31]:
# loop over tracklets
for i, tracklet in enumerate(tracklets):
    # this part is inspired by kitti object development kit matlab code: computeBox3D
    h, w, l = tracklet.size
    # in velodyne coordinates around zero point and without orientation yet
    trackletBox = np.array([
        [-l / 2, -l / 2, l / 2, l / 2, -l / 2, -l / 2, l / 2, l / 2],
        [w / 2, -w / 2, -w / 2, w / 2, w / 2, -w / 2, -w / 2, w / 2],
        [0.0, 0.0, 0.0, 0.0, h, h, h, h]
    ])
    # loop over all data in tracklet
    for translation, rotation, state, occlusion, truncation, amtOcclusion, amtBorders, absoluteFrameNumber in tracklet:
        # determine if object is in the image; otherwise continue
        if truncation not in (xmlParser.TRUNC_IN_IMAGE, xmlParser.TRUNC_TRUNCATED):
            continue
        # re-create 3D bounding box in velodyne coordinate system
        yaw = rotation[2]  # other rotations are supposedly 0
        assert np.abs(rotation[:2]).sum() == 0, 'object rotations other than yaw given!'
        rotMat = np.array([
            [np.cos(yaw), -np.sin(yaw), 0.0],
            [np.sin(yaw), np.cos(yaw), 0.0],
            [0.0, 0.0, 1.0]
        ])
        cornerPosInVelo = np.dot(rotMat, trackletBox) + np.tile(translation, (8, 1)).T
        frame_tracklets[absoluteFrameNumber] = frame_tracklets[absoluteFrameNumber] + [cornerPosInVelo]
        frame_tracklets_types[absoluteFrameNumber] = frame_tracklets_types[absoluteFrameNumber] + [tracklet.objectType]

KeyError: 10

In [27]:
tracklet_rects, tracklet_types = load_tracklets_for_frames(len(list(dataset.velo)), './{}/{}_drive_{}_sync/tracklet_labels.xml'.format(date, date, drive))


Parsing tracklet file ./2011_09_26/2011_09_26_drive_0001_sync/tracklet_labels.xml
File contains 15 tracklets
Loaded 15 tracklets.


KeyError: 10

In [8]:
import os
import numpy as np

basedir = os.getcwd()
date = '2011_09_26'
drive = '0001'

xml_path = './{}/{}_drive_{}_sync/tracklet_labels.xml'.format(date, date, drive)



dataset = pykitti.raw(basedir, date, drive, frames=range(0, 50, 5), imformat='cv2')
# imformat='cv2': OpenCV-friendly format (i.e., uint8 with BGR color channel ordering) 

tracklet_rects, tracklet_types = load_tracklets_for_frames(len(list(dataset.velo)), xml_path )

('Parsing tracklet file', './2011_09_26/2011_09_26_drive_0001_sync/tracklet_labels.xml')
('File contains', 15, 'tracklets')
('Loaded', 15, 'tracklets.')


TypeError: iter() returned non-iterator of type 'list'